In [1]:
import requests

base_url = "http://localhost:8000"
path = "/api/events"

get_endpoint = f"{base_url}{path}"
get_endpoint


create_endpoint = f"{base_url}{path}"
create_endpoint

'http://localhost:8000/api/events'

In [12]:
import random

event_count = 10_000

pages = ['/about','/pricing','/contact','/pages']

for i in range(event_count):
    page = random.choice(pages)
    response = requests.post(url=create_endpoint, json={'page':page})
    if response.ok:
        data=response.json()
        print(data, type(data), data.get('items'))
    else:
        print(response.text)

{'page': '/contact', 'updated_at': '2025-04-01T10:47:58.060996Z', 'id': 6, 'description': '', 'time': '2025-04-01T10:47:58.060989Z'} <class 'dict'> None
{'page': '/contact', 'updated_at': '2025-04-01T10:47:58.114277Z', 'id': 7, 'description': '', 'time': '2025-04-01T10:47:58.114268Z'} <class 'dict'> None
{'page': '/contact', 'updated_at': '2025-04-01T10:47:58.164879Z', 'id': 8, 'description': '', 'time': '2025-04-01T10:47:58.164872Z'} <class 'dict'> None
{'page': '/about', 'updated_at': '2025-04-01T10:47:58.214119Z', 'id': 9, 'description': '', 'time': '2025-04-01T10:47:58.214111Z'} <class 'dict'> None
{'page': '/pricing', 'updated_at': '2025-04-01T10:47:58.264223Z', 'id': 10, 'description': '', 'time': '2025-04-01T10:47:58.264214Z'} <class 'dict'> None
{'page': '/pages', 'updated_at': '2025-04-01T10:47:58.315544Z', 'id': 11, 'description': '', 'time': '2025-04-01T10:47:58.315537Z'} <class 'dict'> None
{'page': '/contact', 'updated_at': '2025-04-01T10:47:58.363198Z', 'id': 12, 'descrip

KeyboardInterrupt: 

In [13]:
import sys
from pathlib import Path

In [14]:
src_path=Path("../src").resolve()
sys.path.append(str(src_path))

In [21]:
from sqlmodel import Session, select
from api.db.session import engine 
from api.events.models import EventModel
from pprint import pprint

In [22]:
with Session(engine) as session:
    query=select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    res=session.exec(query).all()
    pprint(res)

[EventModel(id=2473, description='', updated_at=datetime.datetime(2025, 4, 1, 10, 50, 12, 949973, tzinfo=datetime.timezone.utc), time=datetime.datetime(2025, 4, 1, 10, 50, 12, 949962, tzinfo=datetime.timezone.utc), page='/about'),
 EventModel(id=2472, description='', updated_at=datetime.datetime(2025, 4, 1, 10, 50, 12, 882542, tzinfo=datetime.timezone.utc), time=datetime.datetime(2025, 4, 1, 10, 50, 12, 882522, tzinfo=datetime.timezone.utc), page='/pricing'),
 EventModel(id=2471, description='', updated_at=datetime.datetime(2025, 4, 1, 10, 50, 12, 824406, tzinfo=datetime.timezone.utc), time=datetime.datetime(2025, 4, 1, 10, 50, 12, 824395, tzinfo=datetime.timezone.utc), page='/pricing'),
 EventModel(id=2470, description='', updated_at=datetime.datetime(2025, 4, 1, 10, 50, 12, 768156, tzinfo=datetime.timezone.utc), time=datetime.datetime(2025, 4, 1, 10, 50, 12, 768144, tzinfo=datetime.timezone.utc), page='/pricing'),
 EventModel(id=2469, description='', updated_at=datetime.datetime(2025

In [40]:
from timescaledb.hyperfunctions import time_bucket
from datetime import datetime, timedelta, timezone
from sqlalchemy import func


with Session(engine) as session:
    pages = ['/about']#,'/pricing','/contact','/pages']
    bucket=time_bucket("1 day", EventModel.time)
    start=datetime.now(timezone.utc) - timedelta(hours=1)
    end=datetime.now(timezone.utc) + timedelta(hours=1)
    query=(
        select(
            bucket, 
            EventModel.page,
            func.count().label('event_count')
        )
        .where(
            EventModel.page.in_(pages),
            EventModel.time >= start,
            EventModel.time <= end
        )
        .group_by(
            bucket, 
            EventModel.page,
        )
        .order_by(
            bucket,
            EventModel.page
        )
    )
    res=session.exec(query).fetchall()
    compiled_query = query.compile(compile_kwargs={'literal_binds':True})
    pprint(res)
    print(str(compiled_query))

[(datetime.datetime(2025, 4, 1, 0, 0, tzinfo=datetime.timezone.utc), '/about', 613)]
SELECT time_bucket('1 day'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS event_count 
FROM eventmodel 
WHERE eventmodel.page IN ('/about') AND eventmodel.time >= '2025-04-01 10:14:11.160410+00:00' AND eventmodel.time <= '2025-04-01 12:14:11.160423+00:00' GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page


In [7]:
from pprint import pprint
res = requests.get(url=get_endpoint, params={'duration':'1 day', 'pages':['/pricing','/contact']})
pprint(res.json())

[{'bucket': '2025-04-01T00:00:00Z', 'count': 618, 'page': '/contact'},
 {'bucket': '2025-04-01T00:00:00Z', 'count': 642, 'page': '/pricing'}]
